# Lecture 6 - 16/04/2024

Lecture 6 will be made up of two parts:
1. Analysing SFR of galaxies,
2. Introducing Galaxy Enviroments.

This notebook will focus on (2), look for Lecture6_1 for more information about point (1).

## 2. Galaxy Enviroments
### Introduction
Galaxies are not uniformely distributed due to gravity, leading to more massive structures (groups and clusters). This is a consequence of the hierarchical universe: smaller structures tend to fond earlier and then merge creating larger ones.

Thus, galaxies can be:
- isolated (Milky Way-like galaxies, with parent halo mass $\sim 10^{12}\, M_\odot$);
- in groups (usually up to 50 galaxies in a diameter of 1 to 2 megaparsecs, with a parent halo mass of approximately $10^{13 - 14}\, M_\odot$);
- in clusters (parent halo mass $> 10^{14}\, M_\odot$).

We can see the # of this structures as a function of the dark matter halo mass. This function is called the Halo Mass function and highlights that there are few clusters compared to groups and few groups compared to isolated galaxies.

![Halo mass function](halomassf.jpg)

The most massive galaxy of a given halo is called the central galaxy, all the others orbiting in the same halo are called satellites. An isolated galaxy is by definition central.

But how can we define the enviroment without seeing dm?
The 2 most used ways are:
- counting galaxies in a cylinder with a base radius $r_{ap}$ (in Kpc) and a depth of $\Delta \nu$ (in km/s), the densest environments will have the highest scores in this metric.
- finding the distance of the Nth nearest neighbour to each galaxy, the densest environments will have the lowest score in this metric.

The first method is the most popular. We define a surface density as:
$$\Sigma_{r_{ap}} = \frac{\Sigma_{gal} \, \omega_{gal}}{\pi \times r_{ap}^2} \, ,$$

where $\omega_{gal}$ is the weight of each galaxy, usually equal to 1.

But why do we use a surface density and not a volumetric one? 
1. measuring $\Delta \nu$ is hard and we might have to increase it in order to include the structure we're considering,
2. we might want to compare two cylinders with same $r_{ap}$ but different depths,
3. by considering a volume, the depth would dominate over the aperture.


Both $r_{ap}$ and $\Delta \nu$ must be chosen wisely: too big of a radius gives us high knowledge of clusters but few information on groups, this is why $r_{ap} \in (500, 1000)$ kpc in most datasets. 
The virial radius of
- Clusters is 1-2 Mpc,
- Groups is 300-500 Kpc,
- MW like galaxies is 150-200 Kpc.

Peculiar velocities in
- Clusters $\sigma_{l.o.s.}$ ~700-1000 km/s,
- Groups $\sigma_{l.o.s.}$ ~200-500 km/s.
(l.o.s. = line of sight)

![cylindrical apertures](cyl_apertures.jpg)

    The Fingers of God effect is where the galaxy distribution is elongated in redshift space, with an axis of elongation pointed toward the observer. It is caused by a Doppler shift associated with the random peculiar velocities of galaxies bound in structures such as clusters. The large velocities that lead to this effect are associated with the gravity of the cluster by means of the virial theorem; they change the observed redshifts of the galaxies in the cluster. The deviation from the Hubble's law relationship between distance and redshift is altered, and this leads to inaccurate distance measurements. 
[Fingers of God](https://en.wikipedia.org/wiki/Redshift-space_distortions)



The enviromental densities that we can find through the two methods can then be converted into an estimate of the halo mass

![Halo mass conversion](envden_halomass.jpg)

From the plots we can see that there are galaxies which live close to a cluster that can be really close with one another but have low halo mass (see i.e. the region with mass $\in (10, 12)$ and percentage rank $\in (80, 100)$).

In [1]:
import numpy as np
from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import FK5, SkyCoord
#from astropy.cosmology import Planck18 as cosmo
from astropy import units as u
from tqdm import tqdm

In [2]:
data = Table.read('SDSS_LAB2024_photometry.fits')

When computing the local environment it is common to define: A Primary galaxy (the one you are computing the local density for), the Neighbours sample (all the other galaxies that contribute to the environment metrics)

In [ ]:
Nneigh = np.zeros_like(data[ra])
AP_radius = 500 #kpc
DV_cut    = 1000 #km/s

NEIcoords = SkyCoord(ra=data['ra']*u.deg, dec=data['dec']*u.deg)

for gal in tqdm(range(len(data['id']))):

    PRIcoords = SkyCoord(ra=data['ra'][gal]*u.deg, dec=data['dec'][gal]*u.deg)
    PRIredshift = data['redshift'][gal]
    
    #Obtain a separation of all galaxies from the primary and convert to arcsec
    Separation = NEIcoords.separation(PRIcoords).to(u.arcsec)

    #Kpc to arcsec and obtain a Separation in Kpc
    arcsec_per_kpc = cosmo.arcsec_per_kpc_proper(PRIredshift)
    Separation_kpc = Separation.value/arcsec_per_kpc.value

    #Evaluate separation in redshift space
    deltav = np.abs(3e5*(data['redshift']-PRIredshift)/(1+PRIredshift))

    #Define the selection conditions
    condition =  (Separation_kpc>0) & (Separation_kpc<AP_radius) & (deltav<DV_cut)
    
    #Save the number of galaxies satisfying the selections. Please note these values do not include the primary galaxy.
    Nneigh[gal] = condition.sum()


In [ ]:
#Turn the number of galaxies into a surface density of galaxies.
Dens_05 = Nneigh/(np.pi*0.5**2)

In [ ]:
tabout = Table((data['id'], Dens_05), names=('id', 'dens_05'))
tabout.write('SDSS_env.fits', overwrite=True)